## 准备数据

In [4]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [5]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [6]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        self.W1=tf.Variable(shape=[28*28,200],dtype=tf.float32,initial_value=tf.random.normal([28*28,200],stddev=0.01))
        self.b1=tf.Variable(shape=[200],dtype=tf.float32,initial_value=tf.zeros([200]))
        self.W2=tf.Variable(shape=[200,10],dtype=tf.float32,initial_value=tf.random.normal([200,10],stddev=0.01))
        self.b2=tf.Variable(shape=[10],dtype=tf.float32,initial_value=tf.zeros([10]))
        ####################
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        x_tmp=tf.reshape(x,[-1,28*28])
        y=tf.tanh(tf.matmul(x_tmp,self.W1)+self.b1)
        logits=tf.matmul(y,self.W2)+self.b2
        ####################
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [7]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [8]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.3034275 ; accuracy 0.09803333
epoch 1 : loss 2.303007 ; accuracy 0.10245
epoch 2 : loss 2.3025866 ; accuracy 0.10651667
epoch 3 : loss 2.3021665 ; accuracy 0.110866666
epoch 4 : loss 2.3017466 ; accuracy 0.11556666
epoch 5 : loss 2.301327 ; accuracy 0.12045
epoch 6 : loss 2.3009074 ; accuracy 0.12565
epoch 7 : loss 2.3004878 ; accuracy 0.13065
epoch 8 : loss 2.3000681 ; accuracy 0.13606666
epoch 9 : loss 2.299649 ; accuracy 0.14171667
epoch 10 : loss 2.2992291 ; accuracy 0.1472
epoch 11 : loss 2.2988095 ; accuracy 0.1526
epoch 12 : loss 2.29839 ; accuracy 0.15908334
epoch 13 : loss 2.29797 ; accuracy 0.16553333
epoch 14 : loss 2.29755 ; accuracy 0.1729
epoch 15 : loss 2.2971296 ; accuracy 0.18048333
epoch 16 : loss 2.2967093 ; accuracy 0.188
epoch 17 : loss 2.2962887 ; accuracy 0.19615
epoch 18 : loss 2.2958677 ; accuracy 0.20408334
epoch 19 : loss 2.2954464 ; accuracy 0.2121
epoch 20 : loss 2.2950246 ; accuracy 0.22088334
epoch 21 : loss 2.2946026 ; accuracy 0.2293666